In [1]:
import numpy as np
import pandas as pd
import math
import sys
from ipynb.fs.full.Formulas import *
from ipynb.fs.full.Environment import *
from ipynb.fs.full.Flow_Grouping_Algo import *
import random

Recall Recall Recall


/run/media/prateek/Documents/joint_optimization_relay_selection/Environment.ipynb:141: RuntimeWarning: invalid value encountered in scalar divide
  "            D2x , D2y = section_formula(vehicle1_x_end,vehicle1_y_end,vehicle2_x_end,vehicle2_y_end,m,n)\n",


In [2]:
Graph = flow_graph_generate()

KeyError: '[(10, 1), (15, 2), 0]'